In [ ]:
import os, sys, timeit, math, copy, random
sys.path.append("../input/d/abhishek/timmmaster")
sys.path.append("../input/ttach003/ttach-0.0.3/")

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torchvision
import cv2
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch import nn, optim
from torch.cuda import amp
import torch.nn.functional as F

import timm

from sklearn.model_selection import StratifiedKFold

import albumentations as A
from albumentations.pytorch import ToTensorV2

import optuna

import transformers

import ttach as tta

In [ ]:
# Config and seed

class Config:
    img_size = 224
    n_splits = 10
    seed = 42
    n_bins = 20 # for validation
    
    device = "cuda"
    batch_size = 64
    num_workers = 2
    
    inference_oof = False
    inference_test = True
    
      
cfg = Config

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #torch.backends.cudnn.deterministic = True

seed_everything(cfg.seed)

In [ ]:
# Dataset

class ImgDataset(Dataset):
    def __init__(self, transform=None, folder = "train"):
        self.transform = transform
        
        # Load labels
        self.df = pd.read_csv(f"../input/petfinder-pawpularity-score/{folder}.csv")
        self.folder = folder
                 
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        id_ = self.df.iloc[idx]["Id"]
        image = cv2.imread(f"../input/petfinder-pawpularity-score/{self.folder}/{id_}.jpg")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float64)
        
        if self.folder == "train":
            label = self.df.iloc[idx]['Pawpularity']
        else:
            label = 0
        
        if self.transform is not None:
            image = self.transform(image = image)["image"]
        
        return id_, image, torch.from_numpy(np.array(label)).float()

test_transform224 = A.Compose([
    A.SmallestMaxSize(224),
    A.CenterCrop(224, 224),
    A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0),
    ToTensorV2()
])

test_transform380 = A.Compose([
    A.SmallestMaxSize(380),
    A.CenterCrop(380, 380),
    A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0),
    ToTensorV2()
])

test_transform384 = A.Compose([
    A.SmallestMaxSize(384),
    A.CenterCrop(384, 384),
    A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0),
    ToTensorV2()
])

val_dataset224 = ImgDataset(transform = test_transform224, folder= "train")
test_dataset224 = ImgDataset(transform = test_transform224, folder= "test")

val_dataset380 = ImgDataset(transform = test_transform380, folder= "train")
test_dataset380 = ImgDataset(transform = test_transform380, folder= "test")

val_dataset384 = ImgDataset(transform = test_transform384, folder= "train")
test_dataset384 = ImgDataset(transform = test_transform384, folder= "test")

In [ ]:
# Validation

def validation():
    df = pd.read_csv(f"../input/petfinder-pawpularity-score/train.csv")
    labels = pd.cut(np.array(df["Pawpularity"]), bins=cfg.n_bins, labels=False)

    splitter = StratifiedKFold(n_splits = cfg.n_splits, shuffle = True, random_state = cfg.seed)
    splits = splitter.split(labels, labels)
    return splits

In [ ]:
# Modelling

class BaseSwin(nn.Module):
    def __init__(self, name="swin_large_patch4_window7_224", pretrained=True, n_classes = 1):
        super(BaseSwin, self).__init__()
        self.model = timm.create_model(name, pretrained=pretrained)
        self.n_classes = n_classes
        # self.model.head = nn.Sequential(nn.Dropout(p =  dropout), nn.Linear(self.model.head.in_features, 1))
        self.model.head = nn.Linear(self.model.head.in_features, n_classes)

    def forward(self, x):
        x = self.model(x)
        if self.n_classes == 1:
            return x.ravel()
        else:
            return x
        
class BaseCNN(nn.Module):
    def __init__(self, name="ig_resnext101_32x8d", pretrained=True, n_classes = 1):
        super(BaseCNN, self).__init__()
        self.model = timm.create_model(name, pretrained=pretrained)
        self.n_classes = n_classes
        self.n_features = self.model.num_features
        self.model.fc = nn.Identity()
        self.fc = nn.Linear(self.n_features, self.n_classes)

    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        if self.n_classes == 1:
            return x.ravel()
        else:
            return x
        
class BaseCNN_effnet(nn.Module):
    def __init__(self, name="tf_efficientnet_b4_ns", pretrained=True, n_classes = 1):
        super(BaseCNN_effnet, self).__init__()
        self.model = timm.create_model(name, pretrained=pretrained)
        self.n_classes = n_classes
        self.n_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.fc = nn.Linear(self.n_features, self.n_classes)

    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        if self.n_classes == 1:
            return x.ravel()
        else:
            return x
    
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

In [ ]:
# Inference

CENTER_20 = torch.arange(100 / 20 / 2, 100, 100 / 20).cuda()
AVERAGE_20 = torch.tensor([ 3.07883817,  8.20731707, 13.16618076, 18.31838565, 23.16625717,
       28.00385109, 32.86976048, 37.88047809, 42.79871795, 47.91766724,
       53.01474201, 58.00314465, 63.04562738, 67.82681564, 72.77844311,
       77.80508475, 82.81818182, 87.72972973, 92.86363636, 99.70186335]).cuda()

def inference_bce_scaled(preds):
    # Inference function transforming model output to predictions
    return sigmoid(preds) * 100

def inference_ce(preds, mode, values):
    """
    mode : "argmax", "average"
    values : "center", "average"
    """
    if mode == "argmax":
        preds_modif = torch.zeros_like(preds)
        preds_modif[torch.arange(preds.shape[0]), torch.argmax(preds, dim=1)] = 1
    elif mode == "average":
        preds_modif = F.softmax(preds, dim=1)

    if values == "center":
        vals = CENTER_20
    elif values == "average":
        vals = AVERAGE_20

    return torch.sum(preds_modif * vals, dim=1)

def inference_oof(model_class, model_params, weights_path, dataset, inference_function, train_df, columnname = "model1"):
    splits = validation()
    
    device = cfg.device
    
    train_df[columnname] = -1
    
    for fold, (train_idx,val_idx) in enumerate(splits):
        print(f"Fold {fold + 1}", "\n")
        
        model = model_class(**model_params)
        model.to(device)
        
        filename = f'{weights_path}/fold_{fold}.pth'
        model.load_state_dict(torch.load(filename, map_location="cuda"))
        model.eval()
        
        model = tta.ClassificationTTAWrapper(model, tta.aliases.hflip_transform(), merge_mode='mean')
        
        val_sampler = SubsetRandomSampler(val_idx)
        dataloader = DataLoader(dataset, batch_size=cfg.batch_size, sampler=val_sampler, num_workers = cfg.num_workers, pin_memory = True)
        
        for id_, X, target in tqdm(dataloader):
            with torch.no_grad():
                X, target = X.to(device), target.to(device)

                output = model(X)
                preds = inference_function(output)
                
                train_df.loc[id_, columnname] = preds.detach().cpu().numpy()
                
def inference_test(model_class, model_params, weights_path, dataset, inference_function, submission_df, columnname = "model1"):
    dataloader = DataLoader(dataset, batch_size=8, num_workers = 2)
    df = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
    df.set_index("Id", inplace = True)
    device = cfg.device
    
    for fold in range(cfg.n_splits):
        df[fold] = -1
        print(f"Fold {fold + 1}", "\n")
        
        model = model_class(**model_params)
        model.to(device)
        
        filename = f'{weights_path}/fold_{fold}.pth'
        model.load_state_dict(torch.load(filename, map_location="cuda"))
        model.eval()
        
        model = tta.ClassificationTTAWrapper(model, tta.aliases.hflip_transform(), merge_mode='mean')
        
        for id_, X, target in tqdm(dataloader):
            with torch.no_grad():
                X, target = X.to(device), target.to(device)

                output = model(X)
                preds = inference_function(output)
                
                df.loc[id_, fold] = preds.detach().cpu().numpy()
    
    columnnames = [x for x in range(cfg.n_splits)]
    submission_df[columnname] = df[columnnames].mean(axis = 1)

In [ ]:
# OOF preds + blend
if cfg.inference_oof:
    train_df = pd.read_csv(f"../input/petfinder-pawpularity-score/train.csv")
    train_df.set_index("Id", inplace = True)
    
    inference_ce_average_center = lambda preds: inference_ce(preds, "average", "center")
    inference_ce_average_average = lambda preds: inference_ce(preds, "average", "average")
    
    """
    # 224 vit model
    params_224 = {"name": "vit_large_patch16_224", "pretrained" : False}
    inference_oof(BaseSwin, params_224, "../input/vit224bce", val_dataset224, inference_bce_scaled, train_df, "224vit")
    
    # 384 model
    params_384 = {"name": "swin_large_patch4_window12_384", "pretrained" : False}
    inference_oof(BaseSwin, params_384, "../input/petfinder3842", val_dataset384, inference_bce_scaled, train_df, "384")
    

    # 224 model
    params_224 = {"name": "swin_large_patch4_window7_224", "pretrained" : False}
    inference_oof(BaseSwin, params_224, "../input/bce-large-1-epoch", val_dataset224, inference_bce_scaled, train_df, "224")
    """
    # 224 xcit model
    params_224 = {"name": "xcit_small_24_p8_224_dist", "pretrained" : False}
    inference_oof(BaseSwin, params_224, "../input/xcit224", val_dataset224, inference_bce_scaled, train_df, "224xcit")
    
    # 224 CE model
    params_224ce = {"name": "swin_large_patch4_window7_224", "pretrained" : False, "n_classes": 20}
    inference_oof(BaseSwin, params_224ce, "../input/petfinder224ce", val_dataset224, inference_ce_average_center, train_df, "224CE_center")
    
    # 384 CE model
    params_384ce = {"name": "swin_large_patch4_window12_384", "pretrained" : False, "n_classes": 20}
    inference_oof(BaseSwin, params_384ce, "../input/384cee", val_dataset384, inference_ce_average_center, train_df, "384CE_center")
    
    
    
    train_df.to_csv("oof_preds.csv")
    
    """
    # Ridge OOF blender
    from sklearn.linear_model import Ridge
    blender = Ridge(alpha = 0.1, fit_intercept = False)
    blender.fit(train_df[["224", "384", "224CE"]], train_df["Pawpularity"])
    coef = blender.coef_
    
    from sklearn.metrics import mean_squared_error
    rmse_mean = mean_squared_error((train_df["224"] + train_df["384"]) / 2, train_df["Pawpularity"], squared = False)
    rmse_blend = mean_squared_error(coef[0]*train_df["224"] + coef[1]*train_df["384"], train_df["Pawpularity"], squared = False)
    
    print(coef, rmse_mean, rmse_blend)
    
    print(train_df[["224", "384", "224CE"]].corr())
    """

In [ ]:
#mean_squared_error(0.25543955 * train_df["224"] + 0.43732183 * train_df["384"] + 0.3215622 * train_df["224CE"], train_df["Pawpularity"], squared = False)
#mean_squared_error(0.25735024 * train_df["224"] + 0.43644603 * train_df["384"] + 0.31605065 * train_df["224CE"], train_df["Pawpularity"], squared = False)
#mean_squared_error(0.28045218 * train_df["224"] + 0.41132276 * train_df["384"] + 0.31828948 * train_df["224CE"], train_df["Pawpularity"], squared = False)
#mean_squared_error(train_df["224CE"], train_df["Pawpularity"], squared = False)

In [ ]:
# Test prediction
if cfg.inference_test:
    submission_df = pd.read_csv(f"../input/petfinder-pawpularity-score/sample_submission.csv")
    submission_df.set_index("Id", inplace = True)
    
    
    # 224 model
    params_224 = {"name": "swin_large_patch4_window7_224", "pretrained" : False}
    inference_test(BaseSwin, params_224, "../input/bce-large-1-epoch", test_dataset224, inference_bce_scaled, submission_df, "224")

    # 384 model
    params_384 = {"name": "swin_large_patch4_window12_384", "pretrained" : False}
    inference_test(BaseSwin, params_384, "../input/petfinder3842", test_dataset384, inference_bce_scaled, submission_df, "384")
    
    inference_ce_average_center = lambda preds: inference_ce(preds, "average", "center")
    inference_ce_average_average = lambda preds: inference_ce(preds, "average", "average")
    
    # 384 CE model
    params_384ce = {"name": "swin_large_patch4_window12_384", "pretrained" : False, "n_classes": 20}
    inference_test(BaseSwin, params_384ce, "../input/384cee", test_dataset384, inference_ce_average_average, submission_df, "384CE")
    
    # 224 vit model
    params_224vit = {"name": "vit_large_patch16_224", "pretrained" : False}
    inference_test(BaseSwin, params_224vit, "../input/vit224bce", test_dataset224, inference_bce_scaled, submission_df, "224vit")
    
    # 384 vit model
    params_384vit = {"name": "vit_base_patch16_384", "pretrained" : False}
    inference_test(BaseSwin, params_384vit, "../input/vit384", test_dataset384, inference_bce_scaled, submission_df, "384vit")
    
    # Resnext model
    params_resnext = {"name": "ig_resnext101_32x8d", "pretrained" : False}
    inference_test(BaseCNN, params_resnext, "../input/resnext", test_dataset224, inference_bce_scaled, submission_df, "resnext")
    
    # Effnet model
    params_effnet = {"name": "tf_efficientnet_b4_ns", "pretrained" : False}
    inference_test(BaseCNN_effnet, params_effnet, "../input/effnet", test_dataset380, inference_bce_scaled, submission_df, "effnet")
    
    submission_df["Pawpularity"] = 0.13080735 * submission_df["224"] + \
                                   0.14383828 * submission_df["384"] + \
                                   0.16241641 * submission_df["384CE"] + \
                                   0.15283374 * submission_df["224vit"] + \
                                   0.11407149 * submission_df["384vit"] +\
                                   0.15194291 * submission_df["resnext"] +\
                                   0.13068022 * submission_df["effnet"]
    submission_df.drop(columns = ["224", "384", "384CE", "224vit", "384vit", "resnext", "effnet"], inplace = True)
    submission_df.to_csv(f'submission.csv')
    
    print(submission_df)